In [2]:
pip install astropy

   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/6.9 MB 1.1 MB/s eta 0:00:07
   - -------------------------------------- 0.2/6.9 MB 1.8 MB/s eta 0:00:04
   - -------------------------------------- 0.3/6.9 MB 1.7 MB/s eta 0:00:04
   - -------------------------------------- 0.3/6.9 MB 1.8 MB/s eta 0:00:04
   -- ------------------------------------- 0.5/6.9 MB 2.0 MB/s eta 0:00:04
   --- ------------------------------------ 0.6/6.9 MB 2.2 MB/s eta 0:00:03
   ---- ----------------------------------- 0.7/6.9 MB 2.1 MB/s eta 0:00:03
   ---- ----------------------------------- 0.8/6.9 MB 2.1 MB/s eta 0:00:03
   ---- ----------------------------------- 0.8/6.9 MB 1.9 MB/s eta 0:00:04
   ----- ---------------------------------- 1.0/6.9 MB 2.0 MB/s eta 0:00:03
   ----- ---------------------------------- 1.0/6.9 MB 1.9 MB/s eta 0:00:04
   ------- -------------------------------- 1.2/6.9 MB 2.1 MB/s eta 0:00:03
   ------- --------

In [43]:
import numpy as np
import pandas as pd
from astropy.stats import jackknife_resampling
from astropy.stats import jackknife_stats
import scipy.stats as st
from scipy.stats import linregress
from scipy.stats import f


In [5]:
data = np.array([1,2,3,4,5,6,7,8,9,0])
data

array([1, 2, 3, 4, 5, 6, 7, 8, 9, 0])

In [15]:
def jack_mean(data):
    n = data.shape[0]
    resamples = np.empty([n,n-1])
    mean = np.empty([n,1])

    for i in range(n):
        resamples[i] = np.delete(data,i)
        mean[i] = np.mean(resamples[i])
    
    rata_jack = np.mean(mean)

    return rata_jack

In [16]:
jack_mean(data)

4.5

In [14]:
np.mean(data)

4.5

In [39]:
def jack_reg(y,x):
    n = len(y)
    b0 = np.empty([n,1])
    b1 = np.empty([n,1])
    new_y = np.empty([n,n-1])
    new_x = np.empty([n,n-1])

    for i in range(n):
        new_y[i] = np.delete(y,i)
        new_x[i] = np.delete(x,i)
        reg = st.linregress(new_x[i],new_y[i])
        b0[i] = reg[0]
        b1[i] = reg[1]
    
    jack_b0 = np.mean(b0)
    jack_b1 = np.mean(b1)

    return jack_b0, jack_b1


In [32]:
df = pd.read_csv("C:/Users/darre/Desktop/ASDOS KOMSTAT/W13/data_w13.csv",sep=';')
df

,interest_rate,home_price
0,10.3,183800
1,10.3,183200
2,10.1,174900
3,9.3,173500
4,8.4,172900
5,7.3,173200
6,8.4,173200
7,7.9,169700
8,7.6,174500
9,7.6,177900


In [34]:
x = df['interest_rate']
y = df['home_price']

In [36]:
lm = st.linregress(x,y)
lm

LinregressResult(slope=-23128.346012961403, intercept=391806.7483798254, rvalue=-0.603464640875043, pvalue=0.013322387021057836, stderr=8167.692582187297, intercept_stderr=66890.98380946202)

In [38]:
lm[0]

-23128.346012961403

In [40]:
jack_reg(y,x)

(-23095.69561792657, 391377.0764756829)

In [55]:
df = pd.read_csv("C:/Users/darre/Desktop/ASDOS KOMSTAT/Data No 2.csv",sep=';')
df

,Nilai,Kelompok
0,95,1
1,86,1
2,76,1
3,78,1
4,87,1
5,81,1
6,100,1
7,4,1
8,76,2
9,69,2


In [77]:
g1 = df['Nilai'][df['Kelompok'] == 1]
g2 = df['Nilai'][df['Kelompok'] == 2]
g3 = df['Nilai'][df['Kelompok'] == 3]

In [82]:
# anova
nk = 3
nt = len(df)
xbar_1 = np.mean(g1)
xbar_2 = np.mean(g2)
xbar_3 = np.mean(g3)
xbar_tot = np.mean(df['Nilai'])
nT = len(g1) + len(g2) + len(g3)

ssw = np.sum((g1-xbar_1)**2) + np.sum((g2-xbar_2)**2) + np.sum((g3-xbar_3)**2)
sst = np.sum((df['Nilai'] - xbar_tot)**2)
ssb = sst - ssw

msb = ssb / (nk - 1)
msw = ssw / (nT-nk)

fstat = msb/msw

dfb = nk - 1
dfw = nT - nk
p_value = 1 - f.cdf(fstat, dfb, dfw)

In [83]:
print("ANOVA")
print(f"Fhitung : ", fstat)
print(f"SSB     : ", ssb)
print(f"SSW     : ", ssw)
print(f"MSB     : ", msb)
print(f"MSW     : ", msw)
print(f"P value : ", p_value)

ANOVA
Fhitung :  2.4505120801452605
SSB     :  3735.434523809523
SSW     :  13719.136904761905
MSB     :  1867.7172619047615
MSW     :  762.1742724867725
P value :  0.1144875868807157


In [108]:
def io_check(input):
    Q1 = np.quantile(input, .25)
    Q3 = np.quantile(input, .75)
    IQR = Q3 - Q1
    ba_q1 = Q1 - 1.5 * IQR
    bb_q1 = Q1 - 3 * IQR
    bb_q3 = Q3 + 1.5 * IQR
    ba_q3 = Q3 + 3 * IQR

    inner_outliers = input[(bb_q1 < input) & (input < ba_q1) | (bb_q3 < input) & (input < ba_q3)]
    return inner_outliers, bb_q1, ba_q1, bb_q3, ba_q3


In [109]:
io_check(df['Nilai'])

(7     4
 19    4
 Name: Nilai, dtype: int64,
 -34.0,
 11.0,
 131.0,
 176.0)